In [10]:
import monitoring
import logging
import time
import importlib
importlib.reload(monitoring)
from monitoring import Timer, ErrorCatcher

# A function to test timing; this one runs successfully.
@Timer()
def test_function():
    logging.info("Test function started")
    time.sleep(2)  # Simulate processing
    logging.info("Test function completed")
    return "Done!"

# A function to test error catching; it deliberately raises an exception.
@ErrorCatcher()
def test_exception():
    logging.info("Test exception function starting")
    # Intentionally raise an error to trigger the error catcher.
    raise ValueError("This is a test error!")

# Run test_function; its execution details will be logged in timing_results.csv and timing.log.
result = test_function()
print(result)

# Run test_exception in a try/except block to catch and print the error.
try:
    test_exception()
except Exception as e:
    print(f"Caught an exception as expected: {e}")

2025-03-20 08:15:36,164 - INFO - Test function started
2025-03-20 08:15:38,170 - INFO - Test function completed
2025-03-20 08:15:38,171 - INFO - Function `test_function` executed in 2.0071 sec, CPU Time: 0.0018 sec, Memory Change: 0.8125 MB, Final Memory: 78.5469 MB
2025-03-20 08:15:38,173 - INFO - Test exception function starting
2025-03-20 08:15:38,173 - ERROR - Function `test_exception` raised an exception: This is a test error!
Traceback (most recent call last):
  File "/Users/sra/files/projects/monitoring/monitoring.py", line 278, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/n3/q8wb235x57n_sx2q14dsngjr0000gn/T/ipykernel_98128/4159784902.py", line 21, in test_exception
    raise ValueError("This is a test error!")
ValueError: This is a test error!
2025-03-20 08:15:38,173 - ERROR - Function `test_exception` raised an exception: This is a test error!
Traceback (most recent call last):
  File "/Users/sra/files/projects/monitoring/mo

Function `test_function` executed in 2.0071 sec, CPU Time: 0.0018 sec, Memory Change: 0.8125 MB, Final Memory: 78.5469 MB
Done!
Caught an exception as expected: This is a test error!


Test the parquet option:

In [13]:
import monitoring
import logging
import time
import importlib
importlib.reload(monitoring)
from monitoring import Timer, ErrorCatcher

# A function to test timing; this one runs successfully.
@Timer(results_format='parquet')
def test_function():
    logging.info("Test function started")
    time.sleep(2)  # Simulate processing
    logging.info("Test function completed")
    return "Done!"

# A function to test error catching; it deliberately raises an exception.
@ErrorCatcher(results_format='parquet')
def test_exception():
    logging.info("Test exception function starting")
    # Intentionally raise an error to trigger the error catcher.
    raise ValueError("This is a test error!")

# Run test_function; its execution details will be logged in timing_results.csv and timing.log.
result = test_function()
print(result)

# Run test_exception in a try/except block to catch and print the error.
try:
    test_exception()
except Exception as e:
    print(f"Caught an exception as expected: {e}")

2025-03-20 08:15:43,139 - INFO - Test function started
2025-03-20 08:15:45,143 - INFO - Test function completed
2025-03-20 08:15:45,145 - INFO - Function `test_function` executed in 2.0056 sec, CPU Time: 0.0022 sec, Memory Change: 0.0156 MB, Final Memory: 110.1406 MB
2025-03-20 08:15:45,155 - INFO - Test exception function starting
2025-03-20 08:15:45,156 - ERROR - Function `test_exception` raised an exception: This is a test error!
Traceback (most recent call last):
  File "/Users/sra/files/projects/monitoring/monitoring.py", line 278, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/n3/q8wb235x57n_sx2q14dsngjr0000gn/T/ipykernel_98128/2047492326.py", line 21, in test_exception
    raise ValueError("This is a test error!")
ValueError: This is a test error!
2025-03-20 08:15:45,156 - ERROR - Function `test_exception` raised an exception: This is a test error!
Traceback (most recent call last):
  File "/Users/sra/files/projects/monitoring/m

Function `test_function` executed in 2.0056 sec, CPU Time: 0.0022 sec, Memory Change: 0.0156 MB, Final Memory: 110.1406 MB
Done!
Caught an exception as expected: This is a test error!


In [14]:
import duckdb

timing_results = duckdb.sql("""
select * from read_parquet('logs/timing_results.parquet')
""").to_df()

timing_results.head()

,Timestamp,UUID,Function Name,Execution Time (s),CPU Time (sec),Memory Change (MB),Final Memory Usage (MB),Arguments,Log Message
0,2025-03-20 08:10:16,c7ebb846-3445-449b-9f1b-83d7064ba24b,test_function,2.008888,0.003663,0.203125,111.046875,"{""args"": [], ""kwargs"": {}}",Function `test_function` executed in 2.0089 se...
1,2025-03-20 08:15:40,bf3d5b72-a9b3-4d5f-b218-582e0370af0e,test_function,2.006686,0.001914,0.062500,80.375000,"{""args"": [], ""kwargs"": {}}",Function `test_function` executed in 2.0067 se...
2,2025-03-20 08:15:45,fce6f8f5-07fd-4829-b9ed-5144ae71bdf4,test_function,2.005627,0.002220,0.015625,110.140625,"{""args"": [], ""kwargs"": {}}",Function `test_function` executed in 2.0056 se...


Test single-process functions or those that use multiprocessing:

Timer:

In [15]:
import time
import multiprocessing
import pandas as pd
from monitoring import Timer
importlib.reload(monitoring)
from monitoring import Timer

# Function to test timing (simulates work with sleep)
@Timer(log_to_console=True, log_to_file=True, results_format="csv", use_multiprocessing=False)
def test_function_single_process(n):
    """Simulate a function that takes time to execute."""
    time.sleep(n)
    return n

@Timer(log_to_console=True, log_to_file=True, results_format="parquet", use_multiprocessing=True)
def test_function_multi_process(n):
    """Simulate a function that takes time to execute in a multiprocessing context."""
    time.sleep(n)
    return n

def run_single_process_test():
    print("\n--- Running Single Process Test ---\n")
    for i in range(1, 4):
        test_function_single_process(i * 0.5)  # Sleep times: 0.5s, 1s, 1.5s

def run_multiprocessing_test():
    print("\n--- Running Multi-Process Test ---\n")
    pool_size = 4  # Number of parallel processes
    sleep_times = [0.5, 1, 1.5, 2]  # Each process will sleep for different times

    with multiprocessing.Pool(pool_size) as pool:
        pool.map(test_function_multi_process, sleep_times)

def verify_csv_results():
    """Read and verify CSV results."""
    print("\n--- Verifying CSV Results ---\n")
    results_file = "logs/timing_results.csv"
    
    try:
        df = pd.read_csv(results_file)
        print(df.tail(5))  # Show last few entries
    except FileNotFoundError:
        print("CSV file not found! Test may have failed.")

def verify_parquet_results():
    """Read and verify Parquet results."""
    print("\n--- Verifying Parquet Results ---\n")
    results_file = "logs/timing_results.parquet"
    
    try:
        df = pd.read_parquet(results_file)
        print(df.tail(5))  # Show last few entries
    except FileNotFoundError:
        print("Parquet file not found! Test may have failed.")

if __name__ == "__main__":
    run_single_process_test()     # Test normal function timing
    verify_csv_results()          # Check CSV results
    
    run_multiprocessing_test()    # Test multiprocessing safety
    verify_parquet_results()      # Check Parquet results

    print("\nAll tests completed!")


--- Running Single Process Test ---



2025-03-20 12:20:18,412 - INFO - Function `test_function_single_process` executed in 0.5055 sec, CPU Time: 0.0011 sec, Memory Change: 0.0938 MB, Final Memory: 54.2969 MB


Function `test_function_single_process` executed in 0.5055 sec, CPU Time: 0.0011 sec, Memory Change: 0.0938 MB, Final Memory: 54.2969 MB


2025-03-20 12:20:19,425 - INFO - Function `test_function_single_process` executed in 1.0054 sec, CPU Time: 0.0007 sec, Memory Change: 0.0000 MB, Final Memory: 54.5625 MB


Function `test_function_single_process` executed in 1.0054 sec, CPU Time: 0.0007 sec, Memory Change: 0.0000 MB, Final Memory: 54.5625 MB


2025-03-20 12:20:20,932 - INFO - Function `test_function_single_process` executed in 1.5051 sec, CPU Time: 0.0010 sec, Memory Change: 0.0000 MB, Final Memory: 54.5938 MB


Function `test_function_single_process` executed in 1.5051 sec, CPU Time: 0.0010 sec, Memory Change: 0.0000 MB, Final Memory: 54.5938 MB

--- Verifying CSV Results ---

             Timestamp                                  UUID  \
2  2025-03-20 08:08:14  d223eceb-438d-442b-82a4-93d53933089b   
3  2025-03-20 08:15:38  a880c7c0-71c9-4017-8170-51f52b7b3602   
4  2025-03-20 12:20:18  36e77d75-4e0a-4366-ab47-9bf2bed641fb   
5  2025-03-20 12:20:19  d0cd0ec5-aa0f-4e96-ae06-374eaa788bb3   
6  2025-03-20 12:20:20  aa0fbba7-3271-4c9b-abe5-c1958c7b6dc1   

                  Function Name  Execution Time (s)  CPU Time (sec)  \
2                 test_function            2.008413        0.003843   
3                 test_function            2.007055        0.001811   
4  test_function_single_process            0.505475        0.001093   
5  test_function_single_process            1.005370        0.000691   
6  test_function_single_process            1.505141        0.000978   

   Memory Change (M

Process SpawnPoolWorker-4:
Process SpawnPoolWorker-2:
Process SpawnPoolWorker-3:
Process SpawnPoolWorker-1:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Applications/Anaconda/anaconda3/envs/geo/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Applications/Anaconda/anaconda3/envs/geo/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Applications/Anaconda/anaconda3/envs/geo/lib/python3.11/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/Applications/Anaconda/anaconda3/envs/geo/lib/python3.11/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'test_function_multi_process' on <module '__main__' (built-in)>
  File "/Applications/Anaconda/

KeyboardInterrupt: 